<a href="https://colab.research.google.com/github/rubbybbs/ProbML-book-solution/blob/main/ch03/3.6.9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.6.9
编程实现相关向量机（RVM）的参数更新算法，并采样线性核函数即$K(\mathbf x,\mathbf x_n)=\mathbf x^\top \mathbf x_n$ 在UCI Iris分类数据集上进行训练，分析参数估计的结果。

In [57]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 加载数据集
iris = load_iris()
X = iris.data
y = iris.target

# 将标签编码为-1和1
le = LabelEncoder()
y = le.fit_transform(y)
y = np.where(y > 0, 1, -1)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def linear_kernel(x1, x2):
    return np.inner(x1, x2)

def get_kernel_matrix(X1, X2):
    n_samples1, _ = X1.shape
    n_samples2, _ = X2.shape
    K = np.zeros((n_samples1, n_samples2+1))
    K[:, 0] = 1
    for i in range(n_samples1):
        for j in range(n_samples2):
            K[i, j+1] = linear_kernel(X1[i], X2[j])
    return K

def gaussian_posterior(y, alpha, sigma, K):
    Sigma = np.linalg.inv(np.diag(alpha) + K.T @ np.diag(np.repeat(1 / sigma**2, K.shape[0])) @ K)
    mu = 1 / sigma**2 * (Sigma @ K.T @ y)
    return mu, Sigma

# RVM
max_iters = 20

alpha = np.ones(X_train.shape[0] + 1)
sigma = 1
K = get_kernel_matrix(X_train, X_train)

# Train
for _ in range(max_iters):
    mu, Sigma = gaussian_posterior(y_train, alpha, sigma, K)
    # Update alpha
    gamma = 1 - np.diag(Sigma) * alpha
    alpha = np.clip(gamma / (mu**2), 0, 1e10)
    # Update sigma
    sigma = np.linalg.norm(y_train - K @ mu, 2) / (len(y_train) - np.sum(gamma))

# Check alpha
alpha_1 = alpha[1:]
support_vectors = X_train[alpha_1 < 1e10]
support_vector_indices = np.where(alpha_1 < 1e10)[0]
support_vector_labels = y_train[alpha_1 < 1e10]
support_vector_alphas = alpha_1[alpha_1 < 1e10]

print("Learned support vector indices:")
print(support_vector_indices)
print("Corresponding labels:")
print(support_vector_labels)

print("Learned sigma:")
print(sigma)

# Pred
K_test = get_kernel_matrix(X_test, X_train)
y_pred = np.sign(K_test @ mu)

acc = np.sum(y_pred == y_test) / len(y_pred)
print(f'Test accuracy: {acc}')



Learned support vector indices:
[  0   1   4   8  29  41  44  48  75  84  98 110]
Corresponding labels:
[-1 -1 -1 -1  1 -1  1 -1 -1 -1 -1  1]
Learned sigma:
0.026121286811306526
Test accuracy: 1.0
